# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe5c81189a0>
├── 'a' --> tensor([[ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593]])
└── 'x' --> <FastTreeValue 0x7fe5c81187c0>
    └── 'c' --> tensor([[ 1.0636,  1.7441, -1.1289, -0.3075],
                        [ 1.3092, -0.0622,  1.5990,  0.6852],
                        [-1.0572,  0.0384,  1.6339,  0.9064]])

In [4]:
t.a

tensor([[ 1.1788,  1.3660,  0.6405],
        [-0.6932, -1.4356,  1.2593]])

In [5]:
%timeit t.a

76.7 ns ± 0.786 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe5c81189a0>
├── 'a' --> tensor([[-2.2298, -1.1796,  0.6645],
│                   [ 2.0271,  1.2316,  0.2687]])
└── 'x' --> <FastTreeValue 0x7fe5c81187c0>
    └── 'c' --> tensor([[ 1.0636,  1.7441, -1.1289, -0.3075],
                        [ 1.3092, -0.0622,  1.5990,  0.6852],
                        [-1.0572,  0.0384,  1.6339,  0.9064]])

In [7]:
%timeit t.a = new_value

78.8 ns ± 0.687 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1788,  1.3660,  0.6405],
               [-0.6932, -1.4356,  1.2593]]),
    x: Batch(
           c: tensor([[ 1.0636,  1.7441, -1.1289, -0.3075],
                      [ 1.3092, -0.0622,  1.5990,  0.6852],
                      [-1.0572,  0.0384,  1.6339,  0.9064]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1788,  1.3660,  0.6405],
        [-0.6932, -1.4356,  1.2593]])

In [11]:
%timeit b.a

71.7 ns ± 3 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1446, -1.7037,  0.5065],
               [ 0.4765, -0.4660,  1.0208]]),
    x: Batch(
           c: tensor([[ 1.0636,  1.7441, -1.1289, -0.3075],
                      [ 1.3092, -0.0622,  1.5990,  0.6852],
                      [-1.0572,  0.0384,  1.6339,  0.9064]]),
       ),
)

In [13]:
%timeit b.a = new_value

659 ns ± 8.57 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 7.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

281 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

269 µs ± 9.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe51be681c0>
├── 'a' --> tensor([[[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]],
│           
│                   [[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]],
│           
│                   [[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]],
│           
│                   [[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]],
│           
│                   [[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]],
│           
│                   [[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]],
│           
│                   [[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]],
│           
│                   [[ 1.1788,  1.3660,  0.6405],
│                    [-0.6932, -1.4356,  1.2593]]])
└── 'x' --> <FastTreeValue 0x7fe5c81218e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.1 µs ± 402 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe51bdffbb0>
├── 'a' --> tensor([[ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593],
│                   [ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593],
│                   [ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593],
│                   [ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593],
│                   [ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593],
│                   [ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593],
│                   [ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593],
│                   [ 1.1788,  1.3660,  0.6405],
│                   [-0.6932, -1.4356,  1.2593]])
└── 'x' --> <FastTreeValue 0x7fe51bdff6a0>
    └── 'c' --> tensor([[ 1.0636,  1.7441, -1.1289, -0.3075],
                        [ 1.3092, -0.0622,  1.5990,  0.6852],
                 

In [23]:
%timeit t_cat(trees)

45.1 µs ± 437 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.1 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0636,  1.7441, -1.1289, -0.3075],
                       [ 1.3092, -0.0622,  1.5990,  0.6852],
                       [-1.0572,  0.0384,  1.6339,  0.9064]],
              
                      [[ 1.0636,  1.7441, -1.1289, -0.3075],
                       [ 1.3092, -0.0622,  1.5990,  0.6852],
                       [-1.0572,  0.0384,  1.6339,  0.9064]],
              
                      [[ 1.0636,  1.7441, -1.1289, -0.3075],
                       [ 1.3092, -0.0622,  1.5990,  0.6852],
                       [-1.0572,  0.0384,  1.6339,  0.9064]],
              
                      [[ 1.0636,  1.7441, -1.1289, -0.3075],
                       [ 1.3092, -0.0622,  1.5990,  0.6852],
                       [-1.0572,  0.0384,  1.6339,  0.9064]],
              
                      [[ 1.0636,  1.7441, -1.1289, -0.3075],
                       [ 1.3092, -0.0622,  1.5990,  0.6852],
                       [-1.0572,  0.0384,  1.6339,  0.9064]],

In [26]:
%timeit Batch.stack(batches)

107 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0636,  1.7441, -1.1289, -0.3075],
                      [ 1.3092, -0.0622,  1.5990,  0.6852],
                      [-1.0572,  0.0384,  1.6339,  0.9064],
                      [ 1.0636,  1.7441, -1.1289, -0.3075],
                      [ 1.3092, -0.0622,  1.5990,  0.6852],
                      [-1.0572,  0.0384,  1.6339,  0.9064],
                      [ 1.0636,  1.7441, -1.1289, -0.3075],
                      [ 1.3092, -0.0622,  1.5990,  0.6852],
                      [-1.0572,  0.0384,  1.6339,  0.9064],
                      [ 1.0636,  1.7441, -1.1289, -0.3075],
                      [ 1.3092, -0.0622,  1.5990,  0.6852],
                      [-1.0572,  0.0384,  1.6339,  0.9064],
                      [ 1.0636,  1.7441, -1.1289, -0.3075],
                      [ 1.3092, -0.0622,  1.5990,  0.6852],
                      [-1.0572,  0.0384,  1.6339,  0.9064],
                      [ 1.0636,  1.7441, -1.1289, -0.3075],
                   

In [28]:
%timeit Batch.cat(batches)

194 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

550 µs ± 26.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
